In [1]:
flag = False

In [2]:
if flag:
    print(Run)
flag = True

In [3]:
train   = 100
holdout = 100
canons  = 3
random_state = 1+21+int(100*holdout)+canons*100
num_samples = 5000
dense_layer_length = 100
table = 'figures'
if_exists = 'append'

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

# Verify the shapes of the loaded data
print("Training images shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Testing images shape:", test_images.shape)
print("Testing labels shape:", test_labels.shape)

# Normalize the pixel values (0-255 to 0-1)
train_images = train_images / 255.0
test_images = test_images / 255.0

2025-04-17 06:08:19.378315: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 06:08:19.388190: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744888099.400045    5080 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744888099.403557    5080 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 06:08:19.415659: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Training images shape: (60000, 28, 28)
Training labels shape: (60000,)
Testing images shape: (10000, 28, 28)
Testing labels shape: (10000,)


In [5]:
indices = pd.DataFrame(np.reshape(train_images,(60000,-1))).mean().sort_values(ascending = False)[:100].index

X_train = np.reshape(train_images,(60000,-1))[:train,indices]
X_test  = np.reshape(test_images,(10000,-1))[:holdout,indices]
y_train = train_labels[:train]
y_test  = test_labels[:holdout]

In [6]:

len(X_train),len(y_test)

(100, 100)

In [7]:
from os import chdir
chdir('..')
import sopy as sp

from connect import conn
engine = conn()

In [8]:
Run = pd.Series()

d = pd.DataFrame(X_train, columns = range(1,101)).describe()
ir= pd.DataFrame(X_train, columns = range(1,101))
cl= pd.Series(y_train)


means    = [ d[c]['mean'] for c in d]
sigmas2   =  [ [ 1/1**2 for c in d ] for cl1 in cl.unique() ]


spacings  = [ d[c]['std']/4 for c in d]
lattices = [ np.arange( d[c]['min'], d[c]['max'], d[c]['std']/4 ) for c in d ]

Run['canons'] = canons
Run['holdout'] = holdout
Run['random_state'] = random_state
Run['dense_layer_length'] = dense_layer_length

d = pd.DataFrame(X_train, columns = range(1,101)).describe()
ir= pd.DataFrame(X_train, columns = range(1,101))
cl= pd.Series(y_train)

In [9]:
F = {}
for f in range(10):
    u = sp.vector()
    for index in cl[cl==f].index:
        u += sp.vector().gaussian(a=1, ls = 100*[0], positions = ir.loc[index].values, sigmas = sigmas2[f] , lattices = lattices)
    F[f] = u.mul(1./u.n())
    print(len(u))

2025-04-17 06:08:23.543582: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


13
14
6
11
11
5
11
10
8
11


In [12]:
[[ F[i].dot(F[j]) for i in range(j+1,10)] for j in range(10)]

[[<tf.Tensor: shape=(), dtype=float64, numpy=0.3332932822053197>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.5217392717488214>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.5488547868589938>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.27878760985549733>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.3923220471124642>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.3864182786047475>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.42795496629189406>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.3934946816436474>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.41045795385508793>],
 [<tf.Tensor: shape=(), dtype=float64, numpy=0.48204630487024874>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.48211132387486516>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.6088832613898839>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.6938657410878956>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.5432628381230037>,
  <tf.Tensor: shape=(), dtype=float64, numpy=0.5068483481616997>,
  <t

In [14]:
y_flat_pred = [ [([ v.dot(sp.vector().delta(a=1, positions = x, spacings = spacings , lattices = lattices )) for v in list(F.values()) ])]for x in X_test]

In [15]:
y_pred=[np.argmax(y) for y in y_flat_pred]
pd.DataFrame(np.transpose([[float(yy) for yy in y[0]] for y in y_flat_pred])).describe().T.describe()

,count,mean,std,min,25%,50%,75%,max
count,100.0,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02
mean,10.0,2.085052e-49,6.428544e-49,1.913587e-53,2.292324e-52,1.768770e-51,7.219231e-51,2.035557e-48
std,0.0,1.159091e-48,3.635083e-48,8.527773e-53,7.353682e-52,6.937326e-51,2.003712e-50,1.150458e-47
min,10.0,-1.440950e-55,1.932758e-55,-1.398463e-54,-1.421970e-56,-8.542317e-59,-1.108187e-59,-2.464726e-61
25%,10.0,1.924176e-52,5.592956e-52,1.528222e-56,2.209742e-55,1.779809e-54,1.096576e-53,1.595622e-51
50%,10.0,1.936433e-51,5.535782e-51,3.945000e-55,4.385870e-54,1.959730e-53,2.105294e-52,1.763709e-50
75%,10.0,1.722517e-50,4.508393e-50,2.490886e-54,5.989646e-53,3.616285e-52,2.211553e-51,1.426872e-49
max,10.0,9.861773e-48,3.083477e-47,7.820032e-52,5.753835e-51,5.980874e-50,1.253623e-49,9.761553e-47


In [16]:
accuracy_sop = accuracy_score(y_test, y_pred)
accuracy_sop

0.64

In [17]:
##GEMINI GENERATED THIS CODE

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Define the Single Layer Dense Neural Network (Softmax Regression)
class SingleLayerNN:
    def __init__(self, input_size, output_size, learning_rate=0.01, epochs=1000):
        self.input_size = input_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = np.random.rand(input_size, output_size) - 0.5  # Initialize weights randomly
        self.bias = np.zeros((1, output_size))  # Initialize bias to zeros

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def forward(self, X):
        self.z = np.dot(X, self.weights) + self.bias
        self.y_hat = self.softmax(self.z)
        return self.y_hat

    def backward(self, X, y, y_hat):
        m = X.shape[0]
        dz = y_hat - self.one_hot_encode(y)
        dw = (1/m) * np.dot(X.T, dz)
        db = (1/m) * np.sum(dz, axis=0, keepdims=True)
        return dw, db

    def one_hot_encode(self, y):
        one_hot = np.zeros((len(y), self.output_size))
        one_hot[np.arange(len(y)), y] = 1
        return one_hot

    def update_parameters(self, dw, db):
        self.weights -= self.learning_rate * dw
        self.bias -= self.learning_rate * db

    def train(self, X_train, y_train):
        for epoch in range(self.epochs):
            y_hat = self.forward(X_train)
            dw, db = self.backward(X_train, y_train, y_hat)
            self.update_parameters(dw, db)
            if (epoch + 1) % 100 == 0:
                loss = self.categorical_cross_entropy(self.one_hot_encode(y_train), y_hat)
                #print(f"Epoch {epoch + 1}/{self.epochs}, Loss: {loss:.4f}")

    def predict(self, X_test):
        y_prob = self.forward(X_test)
        y_pred = np.argmax(y_prob, axis=1)
        return y_pred

    def categorical_cross_entropy(self, y_true, y_pred):
        # Add a small epsilon to avoid log(0)
        epsilon = 1e-15
        y_pred_clipped = np.clip(y_pred, epsilon, 1 - epsilon)
        loss = -np.mean(np.sum(y_true * np.log(y_pred_clipped), axis=1))
        return loss

# 5. Initialize and train the neural network
input_size = X_train_scaled.shape[1]  # Number of features (4)
output_size = len(np.unique(y_train))  # Number of classes (3)
learning_rate = 0.1
epochs = 2000

model = SingleLayerNN(input_size, output_size, learning_rate, epochs)
model.train(X_train_scaled, y_train)

# 6. Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# 7. Evaluate the model
accuracy_snn = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {accuracy_snn:.4f}")

# You can also print the classification report for more detailed metrics
#from sklearn.metrics import classification_report
#print("\nClassification Report:")
#print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Test Accuracy: 0.6200
